<a href="https://github.com/optuna/optuna-examples/blob/main/hiplot/plot_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualizing High-dimensional Parameter Relationships

This notebook demonstrates the visualization of `study` in Optuna with [HiPlot](https://facebookresearch.github.io/hiplot/).
The hyperparameters of a neural network trained to classify images are optimized and the resulting study is then visualized using these features.

In [ ]:
# If you run this notebook on Google Colaboratory, uncomment the below to install Optuna and HiPlot.
#! pip install --quiet optuna hiplot

## Preparing the Dataset

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

mnist = fetch_openml(name="Fashion-MNIST", version=1)
classes = list(set(mnist.target))

# For demonstrational purpose, only use a subset of the dataset.
n_samples = 800
data = mnist.data[:n_samples]
target = mnist.target[:n_samples]

x_train, x_valid, y_train, y_valid = train_test_split(data, target)

## Defining the Objective Function

In [ ]:
from sklearn.neural_network import MLPClassifier


def objective(trial):

    clf = MLPClassifier(
        hidden_layer_sizes=tuple(
            [trial.suggest_int("n_units_l{}".format(i), 32, 64) for i in range(3)]
        ),
        learning_rate_init=trial.suggest_float("lr_init", 1e-5, 1e-1, log=True),
    )

    for step in range(100):
        clf.partial_fit(x_train, y_train, classes=classes)
        value = clf.score(x_valid, y_valid)

        # Report intermediate objective value.
        trial.report(value, step)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.TrialPruned()

    return value

## Running the Optimization

In [ ]:
import optuna

optuna.logging.set_verbosity(
    optuna.logging.WARNING
)  # This verbosity change is just to simplify the notebook output.

study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=100)

## Visualizing High-dimensional Parameter Relationships

In [ ]:
import hiplot

hiplot.Experiment.from_optuna(study).display()

### Extermal resources by HiPlot

- HiPlot's official examples are available at https://github.com/facebookresearch/hiplot/tree/main/examples.
- [Hiplot's official page](https://facebookresearch.github.io/hiplot/experiment_settings.html#frontend-rendering-settings) explains how to customize the rendered plot.